In [9]:
import json
import requests as req
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import json

from census import Census
from us import states

# Save config information
api_key = "7824425f0ce43628c75b09e5bfc7c575e283699e"
url = "https://api.census.gov/data/"


In [3]:
# Pull data on businesses and number of establishments

years = ['2010', '2011', '2012', '2013', '2014', '2015']

# Query gets us Number of employees, establishments, size of establishments, year of data collected, annual pay, name of state, for each state
query = "EMP,EMPSZES,EMPSZES_TTL,ESTAB,YEAR,PAYANN,GEO_TTL&for=state:*"


for year in years: 
    # Build query URL
    query_url = url + year + "/cbp?get=" + query + "&key=" + api_key 

    # This should get us all establishments of employment in each state for the 2015 year.
    cbp_response = req.get(query_url)
    cbp_json = cbp_response.json()   
    cbp_df = pd.DataFrame(cbp_json, columns =['EMP','EMPSZES','EMPSZES_TTL','ESTAB','YEAR','PAYANN','GEO_TTL','State'])

cbp_df.head()

,EMP,EMPSZES,EMPSZES_TTL,ESTAB,YEAR,PAYANN,GEO_TTL,State
0,EMP,EMPSZES,EMPSZES_TTL,ESTAB,YEAR,PAYANN,GEO_TTL,state
1,1634391,001,All establishments,98540,2015,67370353,Alabama,01
2,91287,212,Establishments with 1 to 4 employees,48732,2015,3784151,Alabama,01
3,138607,220,Establishments with 5 to 9 employees,20941,2015,4758260,Alabama,01
4,183994,230,Establishments with 10 to 19 employees,13655,2015,6561601,Alabama,01


In [5]:
clean_cbp = cbp_df.drop([0]).reset_index()
cbp_df = pd.DataFrame(clean_cbp, columns=['GEO_TTL','State','ESTAB','EMP','EMPSZES_TTL','PAYANN','YEAR'])
cbp_df.head()

,GEO_TTL,State,ESTAB,EMP,EMPSZES_TTL,PAYANN,YEAR
0,Alabama,01,98540,1634391,All establishments,67370353,2015
1,Alabama,01,48732,91287,Establishments with 1 to 4 employees,3784151,2015
2,Alabama,01,20941,138607,Establishments with 5 to 9 employees,4758260,2015
3,Alabama,01,13655,183994,Establishments with 10 to 19 employees,6561601,2015
4,Alabama,01,9670,292316,Establishments with 20 to 49 employees,10487731,2015


In [7]:
#Let's look at just DC for the past 5 years
DC_cbp_df = cbp_df.loc[cbp_df["GEO_TTL"] == "District of Columbia",:]
DC_cbp_df

,GEO_TTL,State,ESTAB,EMP,EMPSZES_TTL,PAYANN,YEAR
80,District of Columbia,11,22553,513002,All establishments,38445917,2015
81,District of Columbia,11,11203,19639,Establishments with 1 to 4 employees,1766767,2015
82,District of Columbia,11,4077,27176,Establishments with 5 to 9 employees,1915331,2015
83,District of Columbia,11,3069,41833,Establishments with 10 to 19 employees,3019355,2015
84,District of Columbia,11,2367,72323,Establishments with 20 to 49 employees,4945404,2015
85,District of Columbia,11,965,66031,Establishments with 50 to 99 employees,4756409,2015
86,District of Columbia,11,594,89700,Establishments with 100 to 249 employees,6757473,2015
87,District of Columbia,11,182,62790,Establishments with 250 to 499 employees,5082807,2015
88,District of Columbia,11,65,43132,Establishments with 500 to 999 employees,4065601,2015
89,District of Columbia,11,31,90378,"Establishments with 1,000 employees or more",6136770,2015


# Income and Poverty per State

In [10]:


# Save config information
api_key = Census("7824425f0ce43628c75b09e5bfc7c575e283699e", year=2015)


census_data = api_key.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population", 
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,02,Alaska,733375.0,33.4,72515.0,33413.0,72957.0,9.948117,4.265894
1,01,Alabama,4830620.0,38.4,43623.0,24091.0,887260.0,18.367415,4.287172
2,05,Arkansas,2958208.0,37.7,41371.0,22798.0,553644.0,18.715520,3.547519
3,04,Arizona,6641928.0,36.8,50255.0,25848.0,1180690.0,17.776314,4.151084
4,06,California,38421464.0,35.8,61818.0,30318.0,6135142.0,15.968007,4.921939
